In [50]:
import numpy as np

In [51]:
# Simulated data: Word presence in emails (1 = present, 0 = absent)
# Features: [urgent, free, click, meeting]
data = np.array([
    [1, 1, 1, 0, 1],  # urgent, free, click, no meeting → spam
    [0, 0, 0, 1, 0],  # no urgent, no free, no click, meeting → not spam
    [1, 0, 1, 0, 1],  # urgent, no free, click, no meeting → spam
    [0, 0, 0, 1, 0],  # no urgent, no free, no click, meeting → not spam
    [1, 1, 0, 0, 1],  # urgent, free, no click, no meeting → spam
    [0, 0, 1, 1, 0],  # no urgent, no free, click, meeting → not spam
])

In [52]:
X = data[:, :-1]  # Features (first 4 columns)
y = data[:, -1:] # Labels (last column, keep as column vector)


print("Features (urgent, free, click, meeting):", X)
print("Labels (1=spam, 0=not spam):", y.flatten())

Features (urgent, free, click, meeting): [[1 1 1 0]
 [0 0 0 1]
 [1 0 1 0]
 [0 0 0 1]
 [1 1 0 0]
 [0 0 1 1]]
Labels (1=spam, 0=not spam): [1 0 1 0 1 0]


In [53]:
print("Training data:")
print("Features (urgent, free, click, meeting):", X)
print("Labels (1=spam, 0=not spam):", y.flatten())

Training data:
Features (urgent, free, click, meeting): [[1 1 1 0]
 [0 0 0 1]
 [1 0 1 0]
 [0 0 0 1]
 [1 1 0 0]
 [0 0 1 1]]
Labels (1=spam, 0=not spam): [1 0 1 0 1 0]


In [54]:
# Neural Network Architecture: 4 inputs → 3 hidden neurons → 1 output
np.random.seed(42)  # For reproducibility

In [55]:
# Layer 1: Input to Hidden (4 inputs, 3 hidden neurons)
weights_hidden = np.random.rand(4, 3) * 0.5
bias_hidden = np.random.rand(1, 3) * 0.5

# Layer 2: Hidden to Output (3 hidden neurons, 1 output)
weights_output = np.random.rand(3, 1) * 0.5
bias_output = np.random.rand(1, 1) * 0.5

In [56]:

def sigmoid(z):
    return 1 / (1 + np.exp(-np.clip(z, -500, 500)))  # Clip to prevent overflow

def sigmoid_derivative(z):
    return z * (1 - z)



In [57]:
learning_rate = 0.5
print("\nTraining the neural network...")

# Training loop
for epoch in range(1000):
    # Forward propagation
    # Hidden layer
    hidden_input = np.dot(X, weights_hidden) + bias_hidden
    hidden_output = sigmoid(hidden_input)
    
    # Output layer
    output_input = np.dot(hidden_output, weights_output) + bias_output
    final_output = sigmoid(output_input)
    
    # Calculate error
    error = y - final_output
    
    # Backward propagation
    # Output layer gradients
    output_error = error * sigmoid_derivative(final_output)
    
    # Hidden layer gradients
    hidden_error = output_error.dot(weights_output.T) * sigmoid_derivative(hidden_output)
    
    # Update weights and biases
    weights_output += hidden_output.T.dot(output_error) * learning_rate
    bias_output += np.sum(output_error, axis=0, keepdims=True) * learning_rate
    
    weights_hidden += X.T.dot(hidden_error) * learning_rate
    bias_hidden += np.sum(hidden_error, axis=0, keepdims=True) * learning_rate
    
    # Print progress every 200 epochs
    if epoch % 200 == 0:
        loss = np.mean(error**2)
        print(f"Epoch {epoch}, Loss: {loss:.4f}")


print("\nTraining complete!")



Training the neural network...
Epoch 0, Loss: 0.2651
Epoch 200, Loss: 0.0035
Epoch 400, Loss: 0.0013
Epoch 600, Loss: 0.0008
Epoch 800, Loss: 0.0006

Training complete!


In [58]:
# Test the trained network
print("\nTesting on training data:")
hidden_output = sigmoid(np.dot(X, weights_hidden) + bias_hidden)
final_output = sigmoid(np.dot(hidden_output, weights_output) + bias_output)


for i in range(len(X)):
    features = X[i]
    actual = y[i][0]
    predicted = final_output[i][0]
    prediction = "SPAM" if predicted > 0.5 else "NOT SPAM"
    print(f"Email {i+1}: {features} → Actual: {actual}, Predicted: {predicted:.3f} ({prediction})")


Testing on training data:
Email 1: [1 1 1 0] → Actual: 1, Predicted: 0.985 (SPAM)
Email 2: [0 0 0 1] → Actual: 0, Predicted: 0.018 (NOT SPAM)
Email 3: [1 0 1 0] → Actual: 1, Predicted: 0.975 (SPAM)
Email 4: [0 0 0 1] → Actual: 0, Predicted: 0.018 (NOT SPAM)
Email 5: [1 1 0 0] → Actual: 1, Predicted: 0.981 (SPAM)
Email 6: [0 0 1 1] → Actual: 0, Predicted: 0.026 (NOT SPAM)


In [59]:
# Test on new data
print("\nTesting on new emails:")
test_cases = [
    [1, 1, 1, 0],  # urgent, free, click, no meeting
    [0, 0, 0, 1],  # no urgent, no free, no click, meeting
    [1, 0, 0, 1],  # urgent, no free, no click, meeting
]

for i, test_input in enumerate(test_cases):
    test_input = np.array(test_input).reshape(1, -1)
    hidden_output = sigmoid(np.dot(test_input, weights_hidden) + bias_hidden)
    prediction = sigmoid(np.dot(hidden_output, weights_output) + bias_output)[0][0]
    result = "SPAM" if prediction > 0.5 else "NOT SPAM"
    print(f"New email {i+1}: {test_input[0]} → {prediction:.3f} ({result})")


Testing on new emails:
New email 1: [1 1 1 0] → 0.985 (SPAM)
New email 2: [0 0 0 1] → 0.018 (NOT SPAM)
New email 3: [1 0 0 1] → 0.198 (NOT SPAM)
